# TensorRT

In [1]:
import pathlib
import os
import time
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

# How to create the docker run directly in Lambda2
# docker run --gpus all -it --mount type=bind,source=/home/jacob-delgado/Documents/CAPSTONE,target=/workspace/CAPSTONE nvcr.io/nvidia/tensorflow:24.10-tf2-py3

2024-11-12 23:54:47.056173: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 23:54:47.320984: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Benchmarking

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
print(trt.trt_utils._pywrap_py_utils.get_linked_tensorrt_version())

(10, 5, 0)


### Benchmarking

In [10]:
data_root = pathlib.Path("../data/Monkeypox_Data/Original_Images")    # points to the folder containing the images that will be used for training
saved_model_dir = '../best_model/model1/best_f1score_fold'
optimized_model_dir = '../tensorRT_model/test'
# optimized_model_dir = '../tensorRT_model/test_INT8'

# hyperparameters
batch_size = 32         # size of the batch that will be fed to model
img_height = 224        # input image height
img_width = 224         # input image width
test_size = 0.2

# Load dataset without splitting
dataset = tf.keras.utils.image_dataset_from_directory(
    data_root,                                  # loads images from the data_root directory
    image_size=(img_height, img_width),         # resizes all images to (224, 224) pixels
    batch_size=batch_size,                      # set the batch size
    shuffle=True                                # shufle data when loaded
)

# normalization_layer = layers.Rescaling(1./255)
# dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

Found 228 files belonging to 2 classes.


### Inference and Metrics Setup

In [3]:
image_batches, labels = [], []
for image_batch, label_batch in dataset:
    image_batches.append(image_batch)
    labels.append(label_batch)

image_batches = np.concatenate(image_batches) # Flatten batches to get all images
labels = np.concatenate(labels)               # Flatten batches to get all labels  
print(f"Total Images: {image_batches.shape[0]} \nTotal Labels: {labels.shape[0]}")

# Split the data into test subset for benchmarking
_, X_test, _, Y_test = train_test_split(image_batches, labels, test_size=test_size, random_state=42)


X_test = X_test / 255.0


2024-11-12 23:45:54.492110: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Total Images: 228 
Total Labels: 228


### Model Size Difference

In [4]:
def get_model_size(model_dir):
    total_size = 0
    for dirpath, _, filenames in os.walk(model_dir):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)  # Convert to MB

# original_size = get_model_size(saved_model_dir)
# optimized_size = get_model_size(optimized_model_dir)
# print(f"Original Model Size: {original_size:.2f} MB")
# print(f"Optimized Model Size: {optimized_size:.2f} MB")
# print(f"Compression Ratio: {original_size / optimized_size:.2f}")


### Run inference on original model


In [10]:
# run inference
trials = 500
inference_times = []

# Load the SavedModel directly
model = tf.saved_model.load('../best_model/model1/best_f1score_fold')

# # Load the SavedModel using TFSMLayer, treating it as a Keras layer
# model_layer = tf.keras.layers.TFSMLayer('../best_model/model1/best_f1score_fold', call_endpoint='serving_default')

# # Wrap the TFSMLayer in a Sequential model for inference
# model = tf.keras.Sequential([model_layer])

# # Wrap the SavedModel for inference
# def model_call(inputs):
#     return model.signatures["serving_default"](inputs)["output_0"]

# # Create a Keras Sequential model
# model = tf.keras.Sequential([
#     tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),  # Replace with your model's input shape
#     tf.keras.layers.Lambda(model_call)
# ])

print(f"Running {trials} inference trials on {len(X_test)} test images...")
for i in range(trials):
    start_time = time.perf_counter()

    # Run inference for a batch of images
    # saved_model.predict(X_test, batch_size=batch_size, verbose=0)
    model.signatures["serving_default"](X_test)  # Perform inference directly

    end_time = time.perf_counter()
    inference_time = end_time - start_time
    inference_times.append(inference_time)

    if i % 50 == 0:
        avg_inference = np.mean(inference_times)  # Average inference time per trial
        print(f"Step {i}: average inference time = {avg_inference:.6f} seconds")
        
    tf.keras.backend.clear_session()
        
# Compute throughput (images per second)
# total_time = np.sum(inference_times)
# throughput = (trials * len(X_test)) / total_time
# print(f"Throughput: {throughput:.2f} images/second")

Running 500 inference trials on 46 test images...


TypeError: Binding inputs to tf.function failed due to `too many positional arguments`. Received args: (array([[[[0.68235296, 0.53333336, 0.49019608],
         [0.6862745 , 0.5372549 , 0.49411765],
         [0.6901961 , 0.5411765 , 0.49803922],
         ...,
         [0.85490197, 0.72156864, 0.68235296],
         [0.85882354, 0.7254902 , 0.6862745 ],
         [0.85882354, 0.7254902 , 0.6862745 ]],

        [[0.68235296, 0.53333336, 0.49019608],
         [0.6862745 , 0.5372549 , 0.49411765],
         [0.6901961 , 0.5411765 , 0.49803922],
         ...,
         [0.85882354, 0.7254902 , 0.6862745 ],
         [0.85882354, 0.7254902 , 0.6862745 ],
         [0.85882354, 0.7254902 , 0.6862745 ]],

        [[0.6862745 , 0.5372549 , 0.49411765],
         [0.6862745 , 0.5372549 , 0.49411765],
         [0.6901961 , 0.5411765 , 0.49803922],
         ...,
         [0.8627451 , 0.7294118 , 0.6901961 ],
         [0.8627451 , 0.7294118 , 0.6901961 ],
         [0.8627451 , 0.7294118 , 0.6901961 ]],

  

### Run Inference on Models

In [7]:
def measure_inference_time(model, X_test, batch_size, trials=500):
    inference_times = []

    print(f"Running {trials} inference trials on {len(X_test)} test images...")
    num_batches = len(X_test) // batch_size

    for i in range(trials):
        start_time = time.perf_counter()

        # Loop over the batches of X_test
        for j in range(num_batches):
            batch_start = j * batch_size
            batch_end = (j + 1) * batch_size
            batch_images = X_test[batch_start:batch_end]  # Get a batch of images
            inputs = tf.convert_to_tensor(batch_images) # Ensure the batch is in tensor format

            # Run inference for the batch
            model.signatures["serving_default"](inputs)  # Perform inference directly
            tf.keras.backend.clear_session()

        end_time = time.perf_counter()
        inference_time = end_time - start_time
        inference_times.append(inference_time)

        if i % 100 == 0:
            avg_inference = np.mean(inference_times)  # Average inference time per trial
            print(f"Step {i}: average inference time = {avg_inference:.6f} seconds")

        tf.keras.backend.clear_session()

    # Compute throughput (images per second)
    total_time = np.sum(inference_times)
    throughput = (trials * len(X_test)) / total_time
    return np.mean(inference_times), throughput

### Run Metrics on Models

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load models
original_model = tf.saved_model.load('../best_model/model1/best_f1score_fold')
# optimized_model = tf.saved_model.load('../tensorRT_model/test_INT8')

def measure_metrics(model, X_test, Y_test, batch_size):
    print(f"Evaluating metrics on {len(X_test)} test images...")
    
    num_batches = len(X_test) // batch_size
    all_predicted_classes = []

    # Loop over the test dataset in batches
    for j in range(num_batches):
        batch_start = j * batch_size
        batch_end = (j + 1) * batch_size
        batch_images = X_test[batch_start:batch_end]
        inputs = tf.convert_to_tensor(batch_images)  # Convert batch to tensor

        # Run predictions
        result = model.signatures["serving_default"](inputs)  # Inference
        prediction_logits = result["output_0"].numpy()  # Extract logits
        probabilities = tf.nn.sigmoid(prediction_logits).numpy()  # Apply sigmoid
        predicted_classes = (probabilities > 0.5).astype(int)  # Threshold for binary classification

        # Collect predictions
        all_predicted_classes.extend(predicted_classes)

    # Handle any remaining images that don't fit evenly in batches
    remaining_samples = len(X_test) % batch_size
    if remaining_samples > 0:
        batch_images = X_test[-remaining_samples:]
        inputs = tf.convert_to_tensor(batch_images)
        result = model.signatures["serving_default"](inputs)
        prediction_logits = result["output_0"].numpy()
        probabilities = tf.nn.sigmoid(prediction_logits).numpy()
        predicted_classes = (probabilities > 0.5).astype(int)
        all_predicted_classes.extend(predicted_classes)

    # Flatten predictions and labels to ensure they are 1D arrays
    all_predicted_classes = np.array(all_predicted_classes).flatten()
    Y_test = np.array(Y_test).flatten()

    # Ensure the number of predictions matches the number of ground truth labels
    if len(all_predicted_classes) != len(Y_test):
        raise ValueError(f"Number of predicted classes ({len(all_predicted_classes)}) "
                         f"does not match the number of ground truth labels ({len(Y_test)}).")

    # Calculate metrics
    accuracy = accuracy_score(Y_test, all_predicted_classes)
    precision = precision_score(Y_test, all_predicted_classes, average="binary")
    recall = recall_score(Y_test, all_predicted_classes, average="binary")
    f1 = f1_score(Y_test, all_predicted_classes, average="binary")

    # Output metrics
    metrics_dict = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

    print(f"Metrics: {metrics_dict}")
    return metrics_dict

In [ ]:
print(tf.__version__)

2.16.1


### Perform Benchmarking

In [11]:
original_size = get_model_size(saved_model_dir)
optimized_size = get_model_size(optimized_model_dir)
print(f"Original Model Size: {original_size:.2f} MB")
print(f"Optimized Model Size: {optimized_size:.2f} MB")
print(f"Compression Ratio: {original_size / optimized_size:.2f}")

# Int8 compression ratio: 1.50, speedup factor: 2.50
# Load models
original_model = tf.saved_model.load(saved_model_dir)
optimized_model = tf.saved_model.load(optimized_model_dir)

# # Measure inference time for both models
# original_avg_time, original_throughput = measure_inference_time(
#     original_model, X_test, batch_size=batch_size
# )
# optimized_avg_time, optimized_throughput = measure_inference_time(
#     optimized_model, X_test, batch_size=batch_size
# )

# # Print results
# print("\nResults:")
# print(f"Original Model - Average Inference Time: {original_avg_time:.6f} seconds")
# print(f"Original Model - Throughput: {original_throughput:.2f} images/second")
# print(f"Optimized Model - Average Inference Time: {optimized_avg_time:.6f} seconds")
# print(f"Optimized Model - Throughput: {optimized_throughput:.2f} images/second")

# # Compute speedup
# speedup_factor = original_avg_time / optimized_avg_time
# print(f"\nSpeedup Factor: {speedup_factor:.2f}")

# Measure metrics for both models
original_metrics = measure_metrics(
    model=original_model, 
    X_test=X_test, 
    Y_test=Y_test, 
    batch_size=batch_size
)

optimized_metrics = measure_metrics(
    model=optimized_model, 
    X_test=X_test, 
    Y_test=Y_test, 
    batch_size=batch_size
)

print("\nComparison of Metrics:")
print(f"{'Metric':<12} {'Original Model':<15} {'Optimized Model':<15}")
print("-" * 42)
for metric in original_metrics.keys():
    print(f"{metric:<12} {original_metrics[metric]:<15.4f} {optimized_metrics[metric]:<15.4f}")

# Highlighting model differences
print("\nSummary of Changes:")
for metric in original_metrics.keys():
    change = optimized_metrics[metric] - original_metrics[metric]
    print(f"{metric:<12} Change: {change:.4f}")

Original Model Size: 18.90 MB
Optimized Model Size: 9.01 MB
Compression Ratio: 2.10


: 

In [ ]:
# # Load the optimized TensorRT model
# saved_model_loaded = tf.saved_model.load('path/to/save/tensorrt_model')
# infer = saved_model_loaded.signatures['serving_default']

# # Example input data (adjust as per your model's input requirements)
# input_tensor = tf.convert_to_tensor(your_input_data)

# # Run inference
# output = infer(input_tensor)

# # Process output as needed